In [1]:
# Program to train simple RNN for each node data and save training results and save each model. 
# Then, keep first model as a baseline model and transfer it to all remaining nodes and train DNN. 
# Similarly, keep second model as a baseline model and transfer it to all remaining nodes and train DNN. 
# Keep doing in this way until each node becomes a baseline model and transferred to the remaining nodes.  

In [2]:
# Importing library

import os
from numpy.random import seed
seed(0)
import tensorflow
from tensorflow import keras
tensorflow.random.set_seed(0)

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
%matplotlib inline

import warnings 
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [3]:
# Reading csv file of all node data.

df_in = pd.read_csv('all_node_data.csv')
df_in.head() 

,node_id,rsrp1,rsrp2,rsrp3,rsrp4,rsrp5,rsrq1,rsrq2,rsrq3,rsrq4,rsrq5,rssi1,rssi2,rssi3,rssi4,rssi5,rtt_avg
0,312,-82,-82,-82,-82,-82,-6,-6,-6,-6,-6,-59,-59,-59,-59,-59,0.035095
1,312,-82,-82,-82,-82,-82,-6,-6,-6,-6,-6,-59,-59,-59,-59,-59,0.035158
2,312,-82,-82,-82,-82,-82,-6,-6,-6,-6,-6,-59,-59,-59,-59,-59,0.034815
3,312,-82,-82,-82,-82,-82,-6,-6,-6,-6,-6,-59,-59,-59,-59,-59,0.034977
4,312,-82,-82,-82,-82,-82,-6,-6,-6,-6,-6,-59,-59,-59,-59,-59,0.034520


In [4]:
len(df_in)

20160

In [5]:
df_in.describe()

,node_id,rsrp1,rsrp2,rsrp3,rsrp4,rsrp5,rsrq1,rsrq2,rsrq3,rsrq4,rsrq5,rssi1,rssi2,rssi3,rssi4,rssi5,rtt_avg
count,20160.000000,20160.000000,20160.000000,20160.000000,20160.000000,20160.000000,20160.000000,20160.000000,20160.000000,20160.000000,20160.000000,20160.000000,20160.000000,20160.000000,20160.000000,20160.000000,20160.000000
mean,1061.414286,-86.258482,-86.258433,-86.261756,-86.262302,-86.260218,-8.598661,-8.592659,-8.598512,-8.588641,-8.601935,-60.486657,-60.497669,-60.495188,-60.505853,-60.492808,0.497264
std,430.870275,12.663468,12.661445,12.667944,12.667860,12.666834,2.098353,2.110247,2.104521,2.101445,2.096943,11.638657,11.645537,11.635232,11.629223,11.641441,3.612247
min,312.000000,-118.000000,-118.000000,-118.000000,-118.000000,-118.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-87.000000,-85.000000,-86.000000,-87.000000,-87.000000,-22.946514
25%,865.000000,-95.000000,-95.000000,-95.000000,-95.000000,-95.000000,-10.000000,-10.000000,-10.000000,-10.000000,-10.000000,-69.000000,-69.000000,-69.000000,-69.000000,-69.000000,0.047390
50%,952.000000,-86.000000,-86.000000,-86.000000,-86.000000,-86.000000,-8.000000,-8.000000,-8.000000,-8.000000,-8.000000,-61.000000,-61.000000,-61.000000,-61.000000,-61.000000,0.054749
75%,1044.000000,-77.000000,-77.000000,-77.000000,-77.000000,-77.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-52.000000,-52.000000,-52.000000,-52.000000,-52.000000,0.062431
max,2564.000000,-48.000000,-48.000000,-48.000000,-48.000000,-49.000000,-3.000000,-4.000000,-3.000000,-4.000000,-3.000000,-19.000000,-19.000000,-19.000000,-20.000000,-19.000000,30.000000


In [6]:
# To print all nodes

nodes = sorted(df_in.node_id.unique().tolist())
print(nodes)

[312, 826, 827, 832, 833, 834, 838, 839, 841, 847, 849, 850, 853, 855, 856, 857, 864, 865, 867, 871, 910, 913, 919, 926, 927, 929, 930, 931, 934, 936, 942, 944, 946, 950, 951, 953, 956, 959, 1016, 1018, 1019, 1020, 1021, 1024, 1030, 1031, 1033, 1037, 1038, 1040, 1041, 1043, 1044, 1045, 1046, 1047, 1063, 1064, 1066, 1067, 1068, 1071, 1072, 1074, 1079, 2560, 2561, 2562, 2563, 2564]


In [7]:
# Total number of nodes

print(len(nodes))

70


In [8]:
# Main program

class MakePrediction:
    
    train_data = []
    test_data = []
    train_label = []
    test_label = []
      
    def __init__(self, node):
        self.node = node
  
    ''' filtering out data which contains node == self.node '''
    def tt_split(self, test_size):

        base_df = df_in[df_in.node_id == self.node]
        train_size = int(len(base_df) * test_size)

        X_train = base_df.iloc[:train_size, 1:-1]
        X_test = base_df.iloc[train_size:, 1:-1]
        y_train = base_df.iloc[:train_size, -1]
        y_test = base_df.iloc[train_size:, -1]

        return X_train, X_test, y_train, y_test
  
    ''' creating data time_step = 2 '''
    def time_step(self, test_size):
        X, Y, x, y = self.tt_split(test_size)
        X, Y, x, y = X.reset_index(drop=True), Y.reset_index(drop=True), x.reset_index(drop=True), y.reset_index(drop=True)

        for i in range(0, len(X) - 1):
            a = list(X.iloc[i, :]) #0, 1,  2
            b = list(X.iloc[i + 1, :]) #1, 2, 3

            c = a + b
            self.train_data.append(c)
    
        for i in range(0, len(x) - 1):
            self.train_label.append(x[i + 1])

        for i in range(0, len(Y) - 1):
            a = list(Y.iloc[i, :])
            b = list(Y.iloc[i + 1, :])
            c = a + b
            self.test_data.append(c)

        for i in range(0, len(y) - 1):
            self.test_label.append(y[i + 1])
    
    ''' converting list to array '''
    def array_converter(self, test_size):
        self.time_step(test_size)
        td = np.array(self.train_data)     # train data
        test_d = np.array(self.test_data)  # test data
        tl = np.array(self.train_label)    # train label
        test_l = np.array(self.test_label) # test label

        return td, test_d, tl, test_l
    
    ''' normalizing data (input and output both) '''
    def normalize(self, test_size):
        min_max_sc = MinMaxScaler()
        min_max_sc_labels = MinMaxScaler()
        X_train, X_test, y_train, y_test = self.array_converter(test_size)

        y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

        X_train = min_max_sc.fit_transform(X_train)
        X_test = min_max_sc.transform(X_test)

        y_train = min_max_sc_labels.fit_transform(y_train)
        y_test = min_max_sc_labels.transform(y_test)

        return X_train, X_test, y_train, y_test
  
    ''' creating RNN model '''    
    def create_RNN(self, hidden_units, dense_units):
        model = keras.models.Sequential()
        model.add(keras.Input(shape=(30, 1)))   # Input shape of RNN = 2 time-steps x 15 i/p columns = 30 = X_train.shape[1]
        model.add(layers.SimpleRNN(hidden_units, activation='relu'))  # Input layer
        model.add(layers.Dense(units=32, activation='relu'))          # Ist hidden layer
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(units=16, activation='relu'))          # 2nd hidden layer
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(units=8, activation='relu'))           # 3rd hidden layer
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(units=dense_units, activation='relu')) # Output layer
        model.compile(loss='mean_squared_error', optimizer='adam')
        return model
    
    ''' training '''
    def training_and_testing(self, model, test_size):
        
        X_train, X_test, y_train, y_test = self.normalize(test_size)
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

        early_stopping = keras.callbacks.EarlyStopping(
                      monitor="val_loss",
                      patience=3,
                      verbose=0.2
                  )

        epochs = 50

        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, 
                          callbacks=[early_stopping], verbose=1, batch_size=8)

        return history, X_train, X_test, y_train, y_test, model
    
    ''' results of TL '''
    def create_result(self, file_name, d, model, test_size):
        history, X_train, X_test, y_train, y_test, model = self.training_and_testing(model, test_size)
        mse, mae = self.losses(model, y_test, X_test)
        mse_train, mae_train = self.losses(model, y_train, X_train)
        epochs = self.cal_epochs(history)
        d['file_name'].append(file_name)
        d['mse_test_{}'.format(test_size)].append(mse)
        d['mse_train_{}'.format(test_size)].append(mse_train)
        d['mae_test_{}'.format(test_size)].append(mae)
        d['mae_train_{}'.format(test_size)].append(mae_train)
        d['no_of_epochs_{}'.format(test_size)].append(epochs)
        return d

    ''' results of trained RNN for each node '''
    def create_result_model(self, model, X_train, y_train, X_test, y_test, history):

        train_mse, train_mae = self.losses(model, y_train, X_train)
        test_mse, test_mae = self.losses(model, y_test, X_test)
        epochs = self.cal_epochs(history)
        return train_mse, train_mae, test_mse, test_mae, epochs
        
    ''' loss function '''
    def losses(self, model, actual, p):
        pred = model.predict(p)
        mse = mean_squared_error(actual, pred) * 100
        mae = mean_absolute_error(actual, pred) * 100
        return mse, mae
    
    ''' loss plot '''
    def plot(self, history):        
        plt.figure(figsize=(10, 5))
        plt.plot(history.history['loss'], label="Train loss")
        plt.plot(history.history['val_loss'], label="Test loss")
        plt.grid('True')
        plt.xlabel('Number of epochs', fontsize=14)
        plt.ylabel('Loss (MSE)', fontsize=14)
        plt.legend(['Train', 'Test'], loc='upper right')
        plt.title("Train and Validation loss", fontsize=14)

    ''' calculate epochs '''
    def cal_epochs(self, history):
        return len(history.history['loss'])
    
    ''' save baseline models '''
    def save_model(self, model, path, node):
        model.save(path + '/' + str(node) + '.h5')

#### Saving each baseline model and its training result

In [9]:
# Path to save each baseline model (70 models).

path = r'baseline_models' 

dictn_model = {
    'file_name': [],
    'train_mse': [],
    'train_mae': [],
    'test_mse': [],
    'test_mae': [],
    'no_of_epochs': []
}

for n in nodes:
    print("Node: ", n)
    mp = MakePrediction(n) 
    model = mp.create_RNN(64, 1)  # Neurons in input and output layer
    history, X_train, X_test, y_train, y_test, model = mp.training_and_testing(model, 0.8)
    train_mse, train_mae, test_mse, test_mae, epochs = mp.create_result_model(model, X_train, y_train, X_test, y_test, history)
    
    file_name = str(n) + "_result"
    dictn_model['file_name'].append(file_name)
    dictn_model['train_mse'].append(train_mse)
    dictn_model['train_mae'].append(train_mae)
    dictn_model['test_mse'].append(test_mse)
    dictn_model['test_mae'].append(test_mae)
    dictn_model['no_of_epochs'].append(epochs)
    
    mp.save_model(model, path, n)
    print("*" * 125)
    print()

Node:  312
Epoch 1/50
29/29 [==============================] - 3s 25ms/step - loss: 0.0078 - val_loss: 0.0028
Epoch 2/50
29/29 [==============================] - 0s 10ms/step - loss: 0.0066 - val_loss: 0.0015
Epoch 3/50
29/29 [==============================] - 0s 9ms/step - loss: 0.0062 - val_loss: 0.0012
Epoch 4/50
29/29 [==============================] - 0s 10ms/step - loss: 0.0063 - val_loss: 0.0012
Epoch 5/50
29/29 [==============================] - 0s 10ms/step - loss: 0.0061 - val_loss: 0.0013
Epoch 6/50
29/29 [==============================] - 0s 10ms/step - loss: 0.0061 - val_loss: 0.0013
Epoch 00006: early stopping
*****************************************************************************************************************************

Node:  826
Epoch 1/50
58/58 [==============================] - 3s 15ms/step - loss: 0.3081 - val_loss: 0.1554
Epoch 2/50
58/58 [==============================] - 1s 9ms/step - loss: 0.2143 - val_loss: 0.1109
Epoch 3/50
58/58 [===============

KeyboardInterrupt: 

In [ ]:
# Dataframe of result of each node.

df = pd.DataFrame(dictn_model)
df.head()

In [ ]:
# Saving result in CSV file.

df.to_csv('rnn_results_all_models.csv')

#### Saving filtered baseline models and performing TL (one node to all nodes)

In [11]:
# lodaing CSV file of RNN results.

df = pd.read_csv('rnn_results_all_models.csv')

In [12]:
# Sorting 'test_mse' in increasing order to decide threshold for filtering nodes/models.

sorted(df.test_mse, reverse=False)

[7.73147117183303e-05,
 8.2059966490782e-05,
 0.000104994639279,
 0.000106411533616,
 0.0001077766776812,
 0.0001085608669264,
 0.0001089279002253,
 0.0001106974716063,
 0.0001108656893979,
 0.0001401710727753,
 0.0001408185381721,
 0.0001411796577252,
 0.0001413471791925,
 0.0001418174956434,
 0.0001422753008982,
 0.0001423106441534,
 0.0001423880383714,
 0.0001424473218584,
 0.0001438463525713,
 0.0001438868276439,
 0.0001443714681593,
 0.0001444419917752,
 0.0001452431287133,
 0.0001459843482667,
 0.0001463200430991,
 0.0001481803986179,
 0.0001482535813243,
 0.0001510961204489,
 0.0001528719667176,
 0.0001529472630322,
 0.0001544998644273,
 0.0001548040475596,
 0.0001576345257259,
 0.1158365560114663,
 0.1255437745088446,
 0.4752643117305074,
 0.5935615100070476,
 0.6691408795177192,
 0.7028134543994977,
 0.7704683668071556,
 0.8377787948604769,
 0.8780296269872491,
 0.8945949963270093,
 0.9117973832531372,
 0.9266633178720296,
 0.9296745348797164,
 0.9482671145898304,
 0.975041459

In [13]:
# To filter out some nodes out of 70 nodes, we set a threshold for test MSE of RNN (say, 0.8 here)
# and compare it with the obtained value of test MSE of each model. If obtained test MSE is
# less than the threshold then we keep only those models/nodes for TL further.

new_dictn_model = []
for idx, i in enumerate(df['test_mse']):
    if i <= 0.8: 
        new_dictn_model.append(df.loc[idx, 'file_name'])

In [14]:
# Total number of available models/nodes after filtering

len(new_dictn_model)

40

In [15]:
# Filtered nodes/models to be used for TL 

new_dictn_model = [int(i.split('_')[0]) for i in new_dictn_model]
print(new_dictn_model)

[312, 827, 833, 839, 841, 853, 857, 864, 865, 867, 871, 910, 913, 919, 926, 927, 929, 930, 931, 934, 936, 942, 944, 946, 950, 951, 953, 956, 959, 1016, 1018, 1019, 1020, 1021, 1024, 1030, 1031, 1033, 1037, 1038]


In [19]:
# Using first baseline model and training DNN for remaining nodes using TL and so on...

# test_sizes = [0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6]  # different splits
test_sizes = [0.65, 0.6]  

def create_dict():
    d = {}

    for i in test_sizes:
        d['file_name'] = []
        d['mse_test_{}'.format(str(i))] = []
        d['mse_train_{}'.format(str(i))] = []
        d['mae_test_{}'.format(str(i))] = []
        d['mae_train_{}'.format(str(i))] = []
        d['no_of_epochs_{}'.format(str(i))] = []
    return d

models_path = "baseline_models"            # path of each filtered baseline model
csv_path  = "TL_result_filtered_models/"   # path to save csv file of TL result of
models = os.listdir(models_path)           # filtered models

# To save TL training results in correct filename format 

for model in sorted(models,key=len):
    model_name = model.split(".")[0]
    d = create_dict()
    node = model.split(".")[0]
    for n in nodes:
        if int(n) in new_dictn_model:
            if int(model_name) != n:
                for ts in test_sizes:
                    print(model, " ", n, " ", ts)
                    mp  = MakePrediction(n)
                    file_name = str(node) + "_to_" + str(n)
                    m = tf.keras.models.load_model(models_path + "/" + model)
                    d = mp.create_result(file_name, d, m, ts) 

    d['file_name'] = set(d['file_name'])

    data = {}

    for key in d.keys():
        data[key] = []

    for key in d.keys():
        for i in d[key]:
            data[key].append(i)

    df = pd.DataFrame(data)

    new_dictn = {}

    num = list(d['file_name'])
    num  = sorted([int(i.split("_to_")[1]) for i in num])

    for key in d.keys():
        new_dictn[key] = []

    for n in num:
        for idx, file in enumerate(df['file_name']):
            if int(file.split("_to_")[1]) == int(n):
                get_data = list(df.iloc[idx, :])
                for i, key in enumerate(new_dictn.keys()):
                    new_dictn[key].append(get_data[i])

    df = pd.DataFrame(new_dictn)

    csv_file_name = str(node) + "_to_" + "all.csv"
    df.to_csv(csv_path + csv_file_name)

312.h5   827   0.65
Epoch 1/50
156/156 [==============================] - 3s 11ms/step - loss: 0.0950 - val_loss: 0.0400
Epoch 2/50
156/156 [==============================] - 1s 9ms/step - loss: 0.0631 - val_loss: 0.0401
Epoch 3/50
156/156 [==============================] - 1s 9ms/step - loss: 0.0554 - val_loss: 0.0270
Epoch 4/50
156/156 [==============================] - 1s 9ms/step - loss: 0.0495 - val_loss: 0.0234
Epoch 5/50
156/156 [==============================] - 1s 9ms/step - loss: 0.0441 - val_loss: 0.0170
Epoch 6/50
156/156 [==============================] - 1s 9ms/step - loss: 0.0429 - val_loss: 0.0187
Epoch 7/50
156/156 [==============================] - 1s 9ms/step - loss: 0.0403 - val_loss: 0.0164
Epoch 8/50
156/156 [==============================] - 1s 9ms/step - loss: 0.0386 - val_loss: 0.0217
Epoch 9/50
156/156 [==============================] - 1s 9ms/step - loss: 0.0365 - val_loss: 0.0144
Epoch 10/50
156/156 [==============================] - 1s 9ms/step - loss: 0.03

513/513 [==============================] - 5s 9ms/step - loss: 0.0527 - val_loss: 5.9293e-05
Epoch 2/50
513/513 [==============================] - 4s 8ms/step - loss: 0.0523 - val_loss: 1.9323e-06
Epoch 3/50
513/513 [==============================] - 4s 8ms/step - loss: 0.0524 - val_loss: 0.0028
Epoch 4/50
513/513 [==============================] - 4s 8ms/step - loss: 0.0520 - val_loss: 0.0042
Epoch 5/50
513/513 [==============================] - 4s 8ms/step - loss: 0.0520 - val_loss: 0.0032
Epoch 00005: early stopping
312.h5   867   0.6
Epoch 1/50
534/534 [==============================] - 5s 8ms/step - loss: 0.0506 - val_loss: 0.0025
Epoch 2/50
534/534 [==============================] - 4s 8ms/step - loss: 0.0500 - val_loss: 3.0132e-05
Epoch 3/50
534/534 [==============================] - 4s 7ms/step - loss: 0.0507 - val_loss: 1.0547e-06
Epoch 4/50
534/534 [==============================] - 4s 7ms/step - loss: 0.0505 - val_loss: 3.9592e-04
Epoch 5/50
534/534 [========================

Epoch 00011: early stopping
312.h5   927   0.65
Epoch 1/50
781/781 [==============================] - 7s 8ms/step - loss: 0.0353 - val_loss: 1.0593e-06
Epoch 2/50
781/781 [==============================] - 6s 7ms/step - loss: 0.0353 - val_loss: 1.0588e-06
Epoch 3/50
781/781 [==============================] - 6s 7ms/step - loss: 0.0351 - val_loss: 7.6558e-04
Epoch 4/50
781/781 [==============================] - 6s 7ms/step - loss: 0.0349 - val_loss: 9.9713e-04
Epoch 5/50
781/781 [==============================] - 6s 7ms/step - loss: 0.0350 - val_loss: 5.3178e-04
Epoch 00005: early stopping
312.h5   927   0.6
Epoch 1/50
802/802 [==============================] - 7s 8ms/step - loss: 0.0345 - val_loss: 1.0284e-06
Epoch 2/50
802/802 [==============================] - 6s 7ms/step - loss: 0.0344 - val_loss: 5.2692e-06
Epoch 3/50
802/802 [==============================] - 6s 7ms/step - loss: 0.0341 - val_loss: 2.3286e-06
Epoch 4/50
802/802 [==============================] - 6s 7ms/step - loss:

1427/1427 [==============================] - 11s 8ms/step - loss: 0.0197 - val_loss: 1.5302e-06
Epoch 8/50
1427/1427 [==============================] - 11s 8ms/step - loss: 0.0197 - val_loss: 1.5302e-06
Epoch 00008: early stopping
312.h5   1018   0.65
Epoch 1/50
1450/1450 [==============================] - 12s 8ms/step - loss: 0.0194 - val_loss: 1.5125e-06
Epoch 2/50
1450/1450 [==============================] - 11s 8ms/step - loss: 0.0193 - val_loss: 1.5125e-06
Epoch 3/50
1450/1450 [==============================] - 11s 8ms/step - loss: 0.0193 - val_loss: 1.5125e-06
Epoch 4/50
1450/1450 [==============================] - 11s 8ms/step - loss: 0.0194 - val_loss: 1.5125e-06
Epoch 00004: early stopping
312.h5   1018   0.6
Epoch 1/50
1472/1472 [==============================] - 13s 8ms/step - loss: 0.0190 - val_loss: 1.4929e-06
Epoch 2/50
1472/1472 [==============================] - 11s 8ms/step - loss: 0.0190 - val_loss: 1.4929e-06
Epoch 3/50
1472/1472 [==============================] - 11

2164/2164 [==============================] - 17s 7ms/step - loss: 0.0229 - val_loss: 1.3552e-06
Epoch 2/50
2164/2164 [==============================] - 16s 7ms/step - loss: 0.0195 - val_loss: 1.3552e-06
Epoch 3/50
2164/2164 [==============================] - 16s 7ms/step - loss: 0.0195 - val_loss: 1.3552e-06
Epoch 4/50
2164/2164 [==============================] - 16s 7ms/step - loss: 0.0195 - val_loss: 1.3552e-06
Epoch 00004: early stopping
826.h5   857   0.6
Epoch 1/50
2186/2186 [==============================] - 17s 7ms/step - loss: 0.0289 - val_loss: 1.3491e-06
Epoch 2/50
2186/2186 [==============================] - 16s 7ms/step - loss: 0.0257 - val_loss: 1.3491e-06
Epoch 3/50
2186/2186 [==============================] - 16s 7ms/step - loss: 0.0257 - val_loss: 1.3491e-06
Epoch 4/50
2186/2186 [==============================] - 16s 7ms/step - loss: 0.0257 - val_loss: 1.3491e-06
Epoch 00004: early stopping
826.h5   864   0.65
Epoch 1/50
2209/2209 [==============================] - 17s 

KeyboardInterrupt: 

#### Performing TL (One node to all nodes)

In [17]:
# Program to use first baseline model and train DNN for remaining nodes using TL and so on...ok.

# test_sizes = [0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6]  # different splits
test_sizes = [0.65, 0.6]   

def create_dict():
    d = {}

    for i in test_sizes:
        d['file_name'] = []
        d['mse_test_{}'.format(str(i))] = []
        d['mse_train_{}'.format(str(i))] = []
        d['mae_test_{}'.format(str(i))] = []
        d['mae_train_{}'.format(str(i))] = []
        d['no_of_epochs_{}'.format(str(i))] = []
    return d

models_path = "baseline_models"        # path of each baseline model
csv_path  = "TL_result_all_models/"    # path to save csv file of TL result 
models = os.listdir(models_path)

# To save TL training results in correct filename format 

for model in sorted(models,key=len):
    model_name = model.split(".")[0]
    d = create_dict()
    node = model.split(".")[0]
    for n in nodes:
        if int(model_name) != n:
            for ts in test_sizes:
                print(model, " ", n, " ", ts)
                mp  = MakePrediction(n)
                file_name = str(node) + "_to_" + str(n)
                m = tf.keras.models.load_model(models_path + "/" + model)  # loading baseline model for TL
                d = mp.create_result(file_name, d, m, ts) 

    d['file_name'] = set(d['file_name'])
    
    data = {}

    for key in d.keys():
        data[key] = []

    for key in d.keys():
        for i in d[key]:
            data[key].append(i)

    df = pd.DataFrame(data)
    
    new_dictn = {}
    
    num = list(d['file_name'])
    num  = sorted([int(i.split("_to_")[1]) for i in num])

    for key in d.keys():
        new_dictn[key] = []

    for n in num:
        for idx, file in enumerate(df['file_name']):
            if int(file.split("_to_")[1]) == int(n):
                get_data = list(df.iloc[idx, :])
                for i, key in enumerate(new_dictn.keys()):
                    new_dictn[key].append(get_data[i])
                
    df = pd.DataFrame(new_dictn)
    
    csv_file_name = str(node) + "_to_" + "all.csv"
    df.to_csv(csv_path + csv_file_name)

312.h5   826   0.65
Epoch 1/50
2206/2206 [==============================] - 18s 8ms/step - loss: 0.0131 - val_loss: 0.0107
Epoch 2/50
2206/2206 [==============================] - 16s 7ms/step - loss: 0.0131 - val_loss: 0.0107
Epoch 3/50
2206/2206 [==============================] - 15s 7ms/step - loss: 0.0131 - val_loss: 0.0107
Epoch 4/50
2206/2206 [==============================] - 16s 7ms/step - loss: 0.0131 - val_loss: 0.0107
Epoch 5/50
2206/2206 [==============================] - 16s 7ms/step - loss: 0.0131 - val_loss: 0.0107
Epoch 6/50
2206/2206 [==============================] - 16s 7ms/step - loss: 0.0131 - val_loss: 0.0107
Epoch 00006: early stopping
312.h5   826   0.6
Epoch 1/50
2227/2227 [==============================] - 17s 7ms/step - loss: 0.0130 - val_loss: 0.0104
Epoch 2/50
2227/2227 [==============================] - 17s 8ms/step - loss: 0.0130 - val_loss: 0.0105
Epoch 3/50
2227/2227 [==============================] - 16s 7ms/step - loss: 0.0130 - val_loss: 0.0105
Epoch 

2718/2718 [==============================] - 21s 8ms/step - loss: 0.0107 - val_loss: 0.0071
Epoch 5/50
2718/2718 [==============================] - 21s 8ms/step - loss: 0.0107 - val_loss: 0.0071
Epoch 6/50
2718/2718 [==============================] - 20s 7ms/step - loss: 0.0107 - val_loss: 0.0071
Epoch 7/50
2718/2718 [==============================] - 20s 7ms/step - loss: 0.0107 - val_loss: 0.0072
Epoch 8/50
2718/2718 [==============================] - 20s 7ms/step - loss: 0.0107 - val_loss: 0.0071
Epoch 9/50
2718/2718 [==============================] - 19s 7ms/step - loss: 0.0107 - val_loss: 0.0071
Epoch 00009: early stopping
312.h5   855   0.65
Epoch 1/50
2741/2741 [==============================] - 19s 7ms/step - loss: 0.0106 - val_loss: 0.0070
Epoch 2/50
2741/2741 [==============================] - 18s 7ms/step - loss: 0.0106 - val_loss: 0.0070
Epoch 3/50
2741/2741 [==============================] - 19s 7ms/step - loss: 0.0106 - val_loss: 0.0070
Epoch 4/50
2741/2741 [==============

3254/3254 [==============================] - 23s 7ms/step - loss: 0.0172 - val_loss: 0.0055
Epoch 3/50
3254/3254 [==============================] - 23s 7ms/step - loss: 0.0172 - val_loss: 0.0054
Epoch 4/50
3254/3254 [==============================] - 23s 7ms/step - loss: 0.0172 - val_loss: 0.0054
Epoch 5/50
3254/3254 [==============================] - 23s 7ms/step - loss: 0.0172 - val_loss: 0.0055
Epoch 6/50
3254/3254 [==============================] - 23s 7ms/step - loss: 0.0172 - val_loss: 0.0056
Epoch 00006: early stopping
312.h5   929   0.65
Epoch 1/50
3277/3277 [==============================] - 25s 7ms/step - loss: 0.0171 - val_loss: 0.0055
Epoch 2/50
3277/3277 [==============================] - 24s 7ms/step - loss: 0.0171 - val_loss: 0.0055
Epoch 3/50
3277/3277 [==============================] - 24s 7ms/step - loss: 0.0171 - val_loss: 0.0055
Epoch 4/50
3277/3277 [==============================] - 24s 7ms/step - loss: 0.0172 - val_loss: 0.0057
Epoch 5/50
3277/3277 [==============

3812/3812 [==============================] - 25s 7ms/step - loss: 0.0147 - val_loss: 0.0043
Epoch 4/50
3812/3812 [==============================] - 25s 7ms/step - loss: 0.0147 - val_loss: 0.0043
Epoch 00004: early stopping
312.h5   959   0.6
Epoch 1/50
3834/3834 [==============================] - 29s 7ms/step - loss: 0.0147 - val_loss: 0.0043
Epoch 2/50
3834/3834 [==============================] - 28s 7ms/step - loss: 0.0146 - val_loss: 0.0042
Epoch 3/50
3834/3834 [==============================] - 28s 7ms/step - loss: 0.0146 - val_loss: 0.0042
Epoch 4/50
3834/3834 [==============================] - 28s 7ms/step - loss: 0.0146 - val_loss: 0.0045
Epoch 5/50
3834/3834 [==============================] - 28s 7ms/step - loss: 0.0146 - val_loss: 0.0042
Epoch 6/50
3834/3834 [==============================] - 28s 7ms/step - loss: 0.0146 - val_loss: 0.0042
Epoch 7/50
3834/3834 [==============================] - 28s 7ms/step - loss: 0.0146 - val_loss: 0.0044
Epoch 8/50
3834/3834 [===============

4080/4080 [==============================] - 30s 7ms/step - loss: 0.0138 - val_loss: 0.0039
Epoch 7/50
4080/4080 [==============================] - 30s 7ms/step - loss: 0.0138 - val_loss: 0.0038
Epoch 00007: early stopping
312.h5   1024   0.6
Epoch 1/50
4102/4102 [==============================] - 31s 7ms/step - loss: 0.0137 - val_loss: 0.0040
Epoch 2/50
4102/4102 [==============================] - 30s 7ms/step - loss: 0.0137 - val_loss: 0.0041
Epoch 3/50
4102/4102 [==============================] - 30s 7ms/step - loss: 0.0137 - val_loss: 0.0041
Epoch 4/50
4102/4102 [==============================] - 30s 7ms/step - loss: 0.0137 - val_loss: 0.0038
Epoch 5/50
4102/4102 [==============================] - 30s 7ms/step - loss: 0.0137 - val_loss: 0.0039
Epoch 6/50
4102/4102 [==============================] - 30s 7ms/step - loss: 0.0137 - val_loss: 0.0038
Epoch 7/50
4102/4102 [==============================] - 30s 7ms/step - loss: 0.0137 - val_loss: 0.0038
Epoch 00007: early stopping
312.h5  

Epoch 1/50
4348/4348 [==============================] - 33s 7ms/step - loss: 0.0129 - val_loss: 0.0070
Epoch 2/50
4348/4348 [==============================] - 33s 8ms/step - loss: 0.0129 - val_loss: 0.0070
Epoch 3/50
4348/4348 [==============================] - 32s 7ms/step - loss: 0.0129 - val_loss: 0.0071
Epoch 4/50
4348/4348 [==============================] - 32s 7ms/step - loss: 0.0129 - val_loss: 0.0070
Epoch 5/50
4348/4348 [==============================] - 32s 7ms/step - loss: 0.0129 - val_loss: 0.0070
Epoch 00005: early stopping
312.h5   1040   0.6
Epoch 1/50
4369/4369 [==============================] - 33s 7ms/step - loss: 0.0129 - val_loss: 0.0105
Epoch 2/50
4369/4369 [==============================] - 32s 7ms/step - loss: 0.0129 - val_loss: 0.0105
Epoch 3/50
4369/4369 [==============================] - 32s 7ms/step - loss: 0.0129 - val_loss: 0.0106
Epoch 4/50
4369/4369 [==============================] - 32s 7ms/step - loss: 0.0129 - val_loss: 0.0105
Epoch 5/50
4369/4369 [===

Epoch 5/50
4839/4839 [==============================] - 36s 7ms/step - loss: 0.0140 - val_loss: 0.0176
Epoch 6/50
4839/4839 [==============================] - 37s 8ms/step - loss: 0.0140 - val_loss: 0.0175
Epoch 7/50
4839/4839 [==============================] - 39s 8ms/step - loss: 0.0140 - val_loss: 0.0176
Epoch 00007: early stopping
312.h5   1068   0.6
Epoch 1/50
4860/4860 [==============================] - 41s 8ms/step - loss: 0.0140 - val_loss: 0.0174
Epoch 2/50
4860/4860 [==============================] - 38s 8ms/step - loss: 0.0140 - val_loss: 0.0174
Epoch 3/50
4860/4860 [==============================] - 36s 7ms/step - loss: 0.0140 - val_loss: 0.0174
Epoch 4/50
4860/4860 [==============================] - 36s 7ms/step - loss: 0.0140 - val_loss: 0.0174
Epoch 5/50
4860/4860 [==============================] - 36s 7ms/step - loss: 0.0140 - val_loss: 0.0174
Epoch 6/50
4860/4860 [==============================] - 36s 7ms/step - loss: 0.0139 - val_loss: 0.0174
Epoch 7/50
4860/4860 [===

Epoch 3/50
5107/5107 [==============================] - 38s 7ms/step - loss: 0.0133 - val_loss: 0.0164
Epoch 4/50
5107/5107 [==============================] - 38s 7ms/step - loss: 0.0133 - val_loss: 0.0164
Epoch 00004: early stopping
312.h5   2561   0.6
Epoch 1/50
5128/5128 [==============================] - 39s 7ms/step - loss: 0.0133 - val_loss: 0.0163
Epoch 2/50
5128/5128 [==============================] - 38s 7ms/step - loss: 0.0132 - val_loss: 0.0163
Epoch 3/50
5128/5128 [==============================] - 38s 7ms/step - loss: 0.0132 - val_loss: 0.0164
Epoch 4/50
5128/5128 [==============================] - 38s 7ms/step - loss: 0.0132 - val_loss: 0.0164
Epoch 5/50
5128/5128 [==============================] - 38s 7ms/step - loss: 0.0133 - val_loss: 0.0171
Epoch 00005: early stopping
312.h5   2562   0.65
Epoch 1/50
5151/5151 [==============================] - 39s 7ms/step - loss: 0.0132 - val_loss: 0.0162
Epoch 2/50
5151/5151 [==============================] - 38s 7ms/step - loss: 0.

5842/5842 [==============================] - 42s 7ms/step - loss: 0.0121 - val_loss: 0.0145
Epoch 3/50
5842/5842 [==============================] - 42s 7ms/step - loss: 0.0121 - val_loss: 0.0145
Epoch 4/50
5842/5842 [==============================] - 42s 7ms/step - loss: 0.0121 - val_loss: 0.0145
Epoch 00004: early stopping
826.h5   856   0.65
Epoch 1/50
5865/5865 [==============================] - 42s 7ms/step - loss: 0.0132 - val_loss: 0.0144
Epoch 2/50
5865/5865 [==============================] - 41s 7ms/step - loss: 0.0120 - val_loss: 0.0144
Epoch 3/50
5865/5865 [==============================] - 41s 7ms/step - loss: 0.0120 - val_loss: 0.0144
Epoch 4/50
5865/5865 [==============================] - 41s 7ms/step - loss: 0.0120 - val_loss: 0.0144
Epoch 00004: early stopping
826.h5   856   0.6
Epoch 1/50
5887/5887 [==============================] - 44s 7ms/step - loss: 0.0131 - val_loss: 0.0144
Epoch 2/50
5887/5887 [==============================] - 42s 7ms/step - loss: 0.0120 - val_lo

KeyboardInterrupt: 

In [ ]:
#done